In [1]:
import csv
import pandas as pd
import re
from konlpy.tag import Okt

nlpy = Okt()
file = open('hashcode_classification2020_train.csv', 'r', encoding='utf-8')
line = csv.reader(file)

#전처리하여 저장
f = open("goodWordsk.txt", 'w')

english=open("English_Dict.txt", 'r')
korean=open("Korean_Dict.txt", 'r')
special=open("Special_Dict.txt", 'r', encoding='utf-8')

#영어사전, 한글사전, 특수문자 사전 불러와서 저장
englishDict=[]
sentences = english.readlines()
for sentence in sentences: #한 문장씩
    englishDict.append(sentence[:-1])

koreanDict=[]
sentences = korean.readlines()
for sentence in sentences: #한 문장씩
    koreanDict.append(sentence[:-1])

specialDict=[]
sentences = special.readlines()
for sentence in sentences: #한 문장씩
    specialDict.append(sentence[:-1])

#각 게시물의 (한글/ 영어/ 특수문자) 중 (한글 사전/ 영어 사전/ 특수문자 사전)에 있는 단어들만 추출
for line_text in line:
    #영어만 추출
    eng=re.sub('[^a-zA-Z]',' ',line_text[0]+line_text[1]).strip()
    eng_temp=eng.split(' ')
    eng_temp = list(filter(None, eng_temp))
    for i in range(len(eng_temp)):
        if eng_temp[i] not in englishDict:
            eng_temp[i]=''
    eng_temp = list(filter(None, eng_temp))
    
    #한글만 추출
    kor=nlpy.nouns(line_text[0]+line_text[1])
    for i in range(len(kor)):
        if kor[i] not in koreanDict:
            kor[i]=''
    kor = list(filter(None, kor))
    
    #특수문자만 추출
    spe = re.sub('[a-zA-Z0-9]',' ',line_text[0]+line_text[1])
    spe = re.sub('[ㄱ-ㅣ가-힣]+',' ',spe)
    special_temp = spe.split(' ')
    special_temp = list(filter(None, special_temp))
    for i in range(len(special_temp)):
        if special_temp[i] not in specialDict:
            special_temp[i]=''
    special_temp = list(filter(None, special_temp))
    
    s=' '.join(eng_temp)+' '+' '.join(kor)+' '+' '.join(special_temp)+','+line_text[2]
    f.write(s)
    f.write('\n')
    
    
#     kor=
file.close()
f.close()
english.close()
korean.close()
special.close()

In [11]:
import pandas
import csv
#전처리한 문자 불러와서 학습시키기
file= open('goodWordsk.txt', 'r')
line = file.readline()
training_data=[]
while line:
    data=[line[:-3],line[-2:-1]]
    training_data.append({'data': data[0], 'flag': data[1]})
    line=file.readline()
print(training_data[0])

training_data=pandas.DataFrame(training_data, columns=['data', 'flag'])
training_data.to_csv("train_data.csv", sep=',', encoding='utf-8')
print(training_data.data.shape)
file.close()

{'data': 'wait sleep wait sleep 차이점 및 차이점 ', 'flag': '3'}
(2592,)


In [19]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer


#GET VECTOR COUNT
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(training_data.data)

#SAVE WORD VECTOR
pickle.dump(count_vect.vocabulary_, open("count_vector.pkl","wb"))

In [20]:
from sklearn.feature_extraction.text import TfidfTransformer

#TRANSFORM WORD VECTOR TO TF IDF
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#SAVE TF-IDF
pickle.dump(tfidf_transformer, open("tfidf.pkl","wb"))

In [21]:
from sklearn.neural_network import MLPClassifier

clf_neural = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1)

X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.01, random_state=17)

clf_neural.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(15,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [22]:
pickle.dump(clf_neural, open("softmax.pkl", "wb"))

In [23]:
predicted = clf_neural.predict(X_test)

for predicted_item, result in zip(predicted, y_test):
    print(predicted_item, ' - ', result)

5  -  5
4  -  4
5  -  5
3  -  2
3  -  3
5  -  5
3  -  1
3  -  3
5  -  3
5  -  5
1  -  1
3  -  3
1  -  2
5  -  5
5  -  5
4  -  4
5  -  5
2  -  1
2  -  2
3  -  3
5  -  5
4  -  4
3  -  3
5  -  4
3  -  3
3  -  2


In [24]:
#LOAD MODEL
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

loaded_vec = CountVectorizer(vocabulary=pickle.load(open("count_vector.pkl", "rb")))
loaded_tfidf = pickle.load(open("tfidf.pkl","rb"))
docs_new=[]
f=open("testwords.txt",'r')
sentences = f.readlines()
for sentence in sentences:
    docs_new.append([sentence[:-1]])
f.close()
f2 = open('output_l.csv', 'w', newline='')
wr = csv.writer(f2)
wr.writerow(['label'])
for docs in docs_new:
    X_new_counts = loaded_vec.transform(docs)
    X_new_tfidf = loaded_tfidf.transform(X_new_counts)
    predicted = clf_neural.predict(X_new_tfidf)
    wr.writerow([predicted[0]])

f2.close()